In [ ]:
%pylab inline

** Definir matrizes de rotação **

In [ ]:
def rotate(axis, value):
    if axis=='x':
        rot = np.array([[1,0,0],[0,cos(value),-sin(value)],[0,sin(value),cos(value)]])
    elif axis == 'y':
        rot = np.array([[cos(value),0,sin(value)],[0,1,0],[-sin(value),0, cos(value)]])
    elif axis == 'z':
        rot = np.array([[cos(value),-sin(value),0],[sin(value),cos(value),0],[0,0,1]])    
    else:
        print ("Erro: axis deve ser uma letra (x, y ou z)")
        return
    return rot

In [ ]:
def pca(data):
    """PCA: Perform PCA using SVD.
     data - MxN matrix of input data
            (M dimensions, N trials)
Returns:
  signals - MxN matrix of projected data
       PC - each column is a principal component
        V - variances of principal components"""
    m,n = data.shape
    # subtract off the mean for each dimension
    mn = mean(data,axis=1)
    data = data - tile(mn[:,newaxis],(1,n))
    # construct the matrix Y
    Y = data.T / sqrt(n-1)
    # SVD does it all
    [u,S,PC] = svd(Y)
    # calculate the variances
    V = S**2
    # project the original data
    signals = PC.dot(data)
    return signals,PC,V

** Geração de dados aleatória **

In [ ]:
# Número de pontos
t = 1000;

In [ ]:
# simulacao gravidade
sigma_z = 0.05
mean_z = 9
signal_z = np.random.randn(t)*sigma_z + mean_z

In [ ]:
plot(signal_z)
axis([0,1000,0,10])

** Simulação de aceleração longitudinal **

Os dados são gerados segundo um processo de Markov de 1ª ordem.

In [ ]:
# simulacao acel longitudinal
accel_M_x = np.array([[0.95, 0.05, 0],
           [0, 0.96, 0.04],
           [0.04, 0, 0.96]])
mean_x = np.array([0, 1, -1])       
sigma_x = np.array([0.1, 0.1, 0.1])

estado = 1
signal_x = np.zeros(t)
estados = np.zeros(t)
for i in range(t):
    signal_x[i] = np.random.randn()*sigma_x[estado]+mean_x[estado]
    aleatorio = np.random.uniform(0,1)
    indices = np.where(aleatorio < np.cumsum(accel_M_x[estado,:]))
    estado = indices[0][0]
    estados[i] = estado

In [ ]:
plot(signal_x)
axis([0,1000,0,10])

In [ ]:
# simulacao acel lateral
accel_M_y = np.array([[0.90, 0.05, 0.05],
           [0.2, 0.8, 0],
           [0.2, 0, 0.8]])
mean_y = np.array([0, 0.4, -0.4])       
sigma_y = np.array([0.01, 0.05, 0.05]) 
estado = 1
signal_y = np.zeros(t)
estados = zeros(t)
for i in range(t):
    signal_y[i] = np.random.randn()*sigma_y[estado]+mean_y[estado]
    aleatorio = np.random.uniform(0,1)
    indices = np.where(aleatorio < np.cumsum(accel_M_y[estado,:]))
    estado = indices[0][0]
    estados[i] = estado

In [ ]:
plot(signal_y)
axis([0,1000,0,10])

In [ ]:
dados = vstack((signal_x,signal_y,signal_z))
plot(range(t), dados[0,:])
plot(range(t), dados[1,:])
plot(range(t), dados[2,:])

** Rotação dos dados **

In [ ]:
Rx = rotate('y',pi/4)
Ry = rotate('x',pi/4)
Rz = rotate('z',pi/4)

In [ ]:
dados_rotated = Rx.dot(Ry.dot(Rz.dot(dados)))
plot(range(t), dados_rotated[0,:])
plot(range(t), dados_rotated[1,:])
plot(range(t), dados_rotated[2,:])

In [ ]:
signals, PC, V = pca(dados_rotated)

In [ ]:
plot(range(t), signals[0,:])
plot(range(t), signals[1,:])
plot(range(t), signals[2,:])
title('recovered points')

In [ ]:
proj_data = -PC.dot(dados_rotated)
plot(range(t), proj_data[0,:])
plot(range(t), proj_data[1,:])
plot(range(t), proj_data[2,:])
title('projected original rotated points');